In [1]:
import SimpleITK as sitk
from scipy.io import loadmat, savemat

def resample(ori_img:sitk.Image, scale) -> sitk.Image:
    
    resamplemethod=sitk.sitkNearestNeighbor
    ori_Size = ori_img.GetSize()
    ori_Spacing = ori_img.GetSpacing()   


    if isinstance(scale, list) and len(scale) == 3:
        target_Size = tuple([int(ori_Size[idx]*scale[idx]) for idx in range(3)])
        target_Spacing = tuple([ori_Spacing[idx]/(target_Size[idx]/ori_Size[idx]) for idx in range(3)])
    else:
        target_Size = tuple([int(size*scale) for size in ori_Size])
        target_Spacing = tuple([ori_Spacing[idx]/(target_Size[idx]/ori_Size[idx]) for idx in range(3)])
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ori_img)  

    resampler.SetSize(target_Size)		
    resampler.SetOutputSpacing(target_Spacing)
    if resamplemethod == sitk.sitkNearestNeighbor:
        resampler.SetOutputPixelType(sitk.sitkUInt8)   # 近邻插值用于mask的，保存uint8
    else:
        resampler.SetOutputPixelType(sitk.sitkFloat32)  # 线性插值用于PET/CT/MRI之类的，保存float32
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))    
    resampler.SetInterpolator(resamplemethod)
    return resampler.Execute(ori_img) 

In [2]:
brain3d_mat = loadmat("brain3D.mat")
brain3d = brain3d_mat['data']
ori_img = sitk.GetImageFromArray(brain3d)
ori_img.SetSpacing((0.5, 0.5, 0.5))

In [3]:
print(ori_img.GetSize())
print(ori_img.GetSpacing())
out_img = resample(ori_img=ori_img, scale=1)
print(out_img.GetSize())
print(out_img.GetSpacing())  

sitk.WriteImage(out_img, 'brain3D_{}.nii.gz'.format(out_img.GetSpacing()[0]))
savemat('brain3D_{}.mat'.format(out_img.GetSpacing()[0]), {'data': sitk.GetArrayFromImage(out_img)}, do_compression=True)

(362, 362, 434)
(0.5, 0.5, 0.5)
(362, 362, 434)
(0.5, 0.5, 0.5)


#### in-plane super resolution 

In [2]:
brain3d_mat = loadmat("brain3D.mat")
brain3d = brain3d_mat['data']
ori_img = sitk.GetImageFromArray(brain3d)
ori_img.SetSpacing((0.5, 0.5, 0.5))

In [31]:
print(ori_img.GetSize())
print(ori_img.GetSpacing())
out_img = resample(ori_img=ori_img, scale=[0.05, 2.5, 2.5])
print(out_img.GetSize())
print(out_img.GetSpacing())  

Spacing = out_img.GetSpacing()
sitk.WriteImage(out_img, 'brain3D_{:.0f}_{}_{}.nii.gz'.format(Spacing[0], Spacing[1], Spacing[2]))
savemat('brain3D_{:.0f}_{}_{}.mat'.format(Spacing[0], Spacing[1], Spacing[2]), {'data': sitk.GetArrayFromImage(out_img)}, do_compression=True)

(362, 362, 434)
(0.5, 0.5, 0.5)
(18, 905, 1085)
(10.055555555555555, 0.2, 0.2)


#### super resolution in Z-direction

In [14]:
brain3d_mat = loadmat("brain3D.mat")
brain3d = brain3d_mat['data']
# ori_img = sitk.GetImageFromArray(brain3d[:, :, 171:191]) # 20240325_Z
start = 280
end = 300
# start = 171
# end = 191
# ori_img = sitk.GetImageFromArray(brain3d[:, :, start:end])
ori_img = sitk.GetImageFromArray(brain3d)
ori_img.SetSpacing((0.5, 0.5, 0.5))

In [15]:
print(ori_img.GetSize())
print(ori_img.GetSpacing())
out_img = resample(ori_img=ori_img, scale=[20,1,1])
print(out_img.GetSize())
print(out_img.GetSpacing())  

sitk.WriteImage(ori_img, '20240410_Z/({}, {})_ori_brain3D_{}_{}.nii.gz'.format(start, end,ori_img.GetSize(), ori_img.GetSpacing()))
sitk.WriteImage(out_img, '20240410_Z/({}, {})_brain3D_{}_{}.nii.gz'.format(start, end, out_img.GetSize(), out_img.GetSpacing()))
savemat('20240410_Z/({}, {})_brain3D_{}_{}.mat'.format(start, end, out_img.GetSize(), out_img.GetSpacing()), {'data': sitk.GetArrayFromImage(out_img)}, do_compression=True)

(362, 362, 434)
(0.5, 0.5, 0.5)
(7240, 362, 434)
(0.025, 0.5, 0.5)
